In [1]:
import pandas as pd
# import matplotlib.pyplot as plt
# %matplotlib inline
# from pandas.tseries.offsets import BDay
# from pandas.tseries.offsets import MonthEnd
# import datetime

In [162]:
df = pd.read_csv("../../../Data/vw_HoldActivity.csv",encoding='latin-1',low_memory=False)
dfincident = pd.read_csv("../../../Data/vw_Incident.csv",encoding='latin-1',low_memory=False)
df.head()

,TicketNumber,ActivityId,RegardingObjectId,HoldDuration,Statuscode,Statecode,TimeZoneRuleVersionNumber,HoldTypeName,Reason,AssignedToGroup,StartTime,EndTime,IsThreeStrikeRule,HoldSubReason,Modified_On,Deleted
0,5-0000000464223,28E0D3AE-0907-E711-80E9-3863BB2ED230,2058B401-7A06-E711-80F3-FC15B428DE24,4620,Completed,Completed,NaN,3rd Party,Trade - Trade Status Hold,Internal,2017-03-12 09:53:03,2017-03-15 14:16:17,0,NaN,2017-03-15 14:16:17,NaN
1,5-0000000418488,79A45B63-3507-E711-80E9-3863BB2ED230,54A3CC57-50FE-E611-80F1-FC15B428DE24,0,Open,Open,NaN,Customer,Price Error,Submitter (Contact),2017-03-12 15:05:55,NaN,0,NaN,2017-03-12 15:05:55,NaN
2,5-0000000402148,241CB04D-3907-E711-80E9-3863BB2ED230,60D07540-BBFA-E611-80F6-FC15B4289B54,120,Completed,Completed,0.0,Customer,PO Number Error,Submitter (Contact),2017-03-12 15:33:56,2017-03-12 17:18:02,0,NaN,2017-03-28 06:15:33,NaN
3,5-0000000463272,996E4A98-5407-E711-80E9-3863BB2ED230,905C98C5-CB05-E711-80F3-FC15B4284D68,3300,Completed,Completed,0.0,Internal,Internal Hold - Team Lead,Submitter (Contact),2017-03-12 18:49:25,2017-03-15 01:46:18,0,NaN,2017-03-15 01:46:18,NaN
4,5-0000000408448,2A1CD3FE-5907-E711-80E9-3863BB2ED230,51E16092-0BFD-E611-80F1-FC15B428DE24,180,Completed,Completed,NaN,Internal,A - Missing information from customer - Confir...,Submitter (Contact),2017-03-12 19:28:00,2017-03-12 22:42:43,0,NaN,2017-03-12 22:42:43,NaN


In [163]:
df["TicketNumber"] = [x.lstrip('5-') for x in df["TicketNumber"]]
df["TicketNumber"] = df["TicketNumber"].astype(int)

dfincident["TicketNumber"] = [x.lstrip('5-') for x in dfincident["TicketNumber"]]
dfincident["TicketNumber"] = dfincident["TicketNumber"].astype(int)

dfincident.head()

,TicketNumber,IncidentId,Created_On,Receiveddate,ResolvedDate,Queue,StatusReason,Priority,ValidCase,BusinessFunction,...,IsMagnumCase,IsSignature,Complexity,Numberofreactivations,NumberofChildIncidents,ParentCase,Referencesystem,StateCode,Isrevenueimpacting,IsSOXCase
0,464394,FD1F79A6-4007-E711-80E9-3863BB2ED230,2017-03-12 16:26:32,2017-03-12 16:15:00,2017-03-12 16:30:24,<VL EOC Enterprise Orders and Adjustments>,Rejected,Normal,1,Signature to Invoice (S2I),...,0,0,Medium,NaN,NaN,NaN,NaN,Resolved,0,1.0
1,464404,57B789CC-4507-E711-80E9-3863BB2ED230,2017-03-12 17:03:23,2017-03-12 00:00:00,2017-03-27 23:48:29,<VL AOC SMS&P Processing>,Problem Solved,Normal,1,Signature to Invoice (S2I),...,0,0,NaN,NaN,NaN,NaN,NaN,Resolved,0,0.0
2,464412,480ACC79-4A07-E711-80E9-3863BB2ED230,2017-03-12 17:36:52,2017-03-12 17:30:00,2017-05-12 09:02:44,<VL EOC Ops Out>,Completed,Normal,1,Signature to Invoice (S2I),...,1,1,Medium,NaN,NaN,NaN,NaN,Resolved,0,1.0
3,464428,F26F878E-5507-E711-80E9-3863BB2ED230,2017-03-12 18:56:21,2017-03-12 00:00:00,2017-03-27 23:52:24,<VL AOC SMS&P Processing>,Completed,Normal,1,Signature to Invoice (S2I),...,0,0,NaN,NaN,NaN,NaN,NaN,Resolved,0,0.0
4,464434,6E7599FE-5C07-E711-80E9-3863BB2ED230,2017-03-12 19:49:25,2017-03-12 19:49:23,2017-03-25 22:00:42,<VL AOC Outbound Shipping>,Problem Solved,Normal,1,Signature to Invoice (S2I),...,1,1,NaN,NaN,NaN,NaN,NaN,Resolved,0,1.0


In [185]:
columns = ["TicketNumber", "HoldDuration", "HoldTypeName", "AssignedToGroup"]
for col in df:
    if col not in columns:
        del df[col]
        
dfdummies = pd.get_dummies(data=df, columns=["HoldTypeName", "AssignedToGroup"])        
dfdummies["HoldCount"] = 1

unique_tickets = pd.DataFrame(df["TicketNumber"].unique().tolist(), columns=["TicketNumber"])

unique_tickets["HoldDuration"] = 0

unique_tickets["HoldTypeName_3rd Party"] = 0
unique_tickets["HoldTypeName_Customer"] = 0
unique_tickets["HoldTypeName_Internal"] = 0

unique_tickets["AssignedToGroup_BPO"] = 0
unique_tickets["AssignedToGroup_CRMT"] = 0
unique_tickets["AssignedToGroup_Internal"] = 0
unique_tickets["AssignedToGroup_Microsoft IT"] = 0
unique_tickets["AssignedToGroup_Ops. Program Manager"] = 0
unique_tickets["AssignedToGroup_Submitter (Contact)"] = 0

unique_tickets["HoldCount"] = 0

for i, row in unique_tickets.iterrows():
    ticket = row["TicketNumber"]
    summed = dfdummies[dfdummies["TicketNumber"]==ticket].sum()

    summed = summed.to_frame().T

    for j in range(len(summed.columns)-1):
        j+=1
        unique_tickets.iloc[i, j]+=summed.iloc[0,j]

dfmerged = dfincident.merge(right=unique_tickets, how="left", on="TicketNumber")


27       32760.0
31       17400.0
34        2640.0
35         960.0
36        1440.0
38        1920.0
42         240.0
45         120.0
46       51802.0
47       28500.0
54        8280.0
57        1980.0
58       15660.0
59        3660.0
60        2880.0
61        1500.0
62         960.0
63        2940.0
67        4020.0
74        1140.0
76       15660.0
82           4.0
88        2880.0
89       20700.0
90        1620.0
96        1400.0
98       30960.0
100       2580.0
101        900.0
103       1260.0
          ...   
36039    23460.0
36041    19560.0
36047     4440.0
36048     4140.0
36049     8160.0
36051     4320.0
36052    29396.0
36054    25920.0
36058    21458.0
36071      360.0
36072    10260.0
36073      313.0
36074     1500.0
36075     2880.0
36077      600.0
36078      480.0
36079    12060.0
36080    28860.0
36081      780.0
36082     1080.0
36084     1085.0
36086     4320.0
36088    11040.0
36090    23160.0
36095    18000.0
36097     1440.0
36101     2580.0
36107    16980

In [187]:
dfmerged.head()
print(dfmerged["HoldCount"][dfmerged["HoldCount"]>0])

27       3.0
31       2.0
34       1.0
35       1.0
36       3.0
38       3.0
39       1.0
42       1.0
45       1.0
46       3.0
47       3.0
54       1.0
57       1.0
58       2.0
59       2.0
60       2.0
61       2.0
62       1.0
63       1.0
67       1.0
74       1.0
76       1.0
82       1.0
88       1.0
89       6.0
90       1.0
96       1.0
98       3.0
100      1.0
101      1.0
        ... 
36047    2.0
36048    2.0
36049    2.0
36050    1.0
36051    2.0
36052    3.0
36054    5.0
36055    1.0
36058    3.0
36071    2.0
36072    2.0
36073    1.0
36074    2.0
36075    3.0
36077    2.0
36078    1.0
36079    2.0
36080    2.0
36081    1.0
36082    1.0
36084    1.0
36086    2.0
36088    3.0
36090    1.0
36095    2.0
36097    1.0
36101    1.0
36107    1.0
36108    1.0
36109    1.0
Name: HoldCount, Length: 12372, dtype: float64


In [168]:
columns = ["TicketNumber", "HoldDuration", "HoldTypeName", "AssignedToGroup"]
for col in df:
    if col not in columns:
        del df[col]
        
dfdummies = pd.get_dummies(data=df, columns=["HoldTypeName", "AssignedToGroup"])        

dfdummies["Count"] = 1

unique_tickets = pd.DataFrame(df["TicketNumber"].unique().tolist(), columns=["TicketNumber"])

# unique_tickets["TicketNumber"] = [x.lstrip('5-') for x in unique_tickets["TicketNumber"]]
# unique_tickets["TicketNumber"] = unique_tickets["TicketNumber"].astype(int)

unique_tickets["HoldDuration"] = 0

unique_tickets["HoldTypeName_3rd Party"] = 0
unique_tickets["HoldTypeName_Customer"] = 0
unique_tickets["HoldTypeName_Internal"] = 0

unique_tickets["AssignedToGroup_BPO"] = 0
unique_tickets["AssignedToGroup_CRMT"] = 0
unique_tickets["AssignedToGroup_Internal"] = 0
unique_tickets["AssignedToGroup_Microsoft IT"] = 0
unique_tickets["AssignedToGroup_Ops. Program Manager"] = 0
unique_tickets["AssignedToGroup_Submitter (Contact)"] = 0

unique_tickets["HoldCount"] = 0

for i, row in unique_tickets.iterrows():
    ticket = row["TicketNumber"]
    summed = dfdummies[dfdummies["TicketNumber"]==ticket].sum()

# tramspose summed and merge it with unique_tickets
#     row["HoldDuration"] = summed["HoldDuration"]
#     row["HoldCount"] = summed["Count"]
#     print(row)
    
    summed = summed.to_frame().T
    
    print(summed.columns)
    print(unique_tickets.columns)
    for j in range(len(summed.columns)-1):
        unique_tickets.iloc[i, j]+=summed.iloc[0,j]
#     merged = unique_tickets.merge(right=summed, how="left", on="TicketNumber")
#     row["HoldTypeName_3rd Party"] = summed["HoldTypeName_3rd Party"]
#     row["HoldTypeName_Customer"] = summed["HoldTypeName_Customer"]
#     row["HoldTypeName_Internal"] = summed["HoldTypeName_Internal"]
    
#     row["AssignedToGroup_Internal"] = summed["AssignedToGroup_Internal"]
#     row["AssignedToGroup_Submitter (Contact)"] = summed["AssignedToGroup_Submitter (Contact)"]
#     row["AssignedToGroup_Ops. Program Manager"] = summed["AssignedToGroup_Ops. Program Manager"]
#     row["AssignedToGroup_Microsoft IT"] = summed["AssignedToGroup_Microsoft IT"]
#     row["AssignedToGroup_BPO"] = summed["AssignedToGroup_BPO"]
#     row["AssignedToGroup_CRMT"] = summed["AssignedToGroup_CRMT"]
#     print(merged.head())
    break

# print(unique_tickets.head())
dfmerged = dfincident.merge(right=unique_tickets, how="left", on="TicketNumber")
print(dfmerged["HoldDuration"][dfmerged["HoldDuration"]>0])

3547      1380.0
4206     14460.0
4264      1260.0
4717     27420.0
4740      1560.0
4782      1080.0
4956      3000.0
5023      4680.0
5100      3300.0
5251       660.0
5432      1368.0
5519       720.0
5614     24240.0
5626      1380.0
5690        60.0
5704     15660.0
5733      8340.0
6273      3096.0
6297       240.0
7124     10996.0
9903      2820.0
9950      2820.0
10119     2640.0
10214    62280.0
10288    11040.0
10417      720.0
10877      900.0
11467     4800.0
11631     2160.0
11712     2700.0
          ...   
20577     4440.0
20939     1080.0
21326    10200.0
21478      960.0
24620     4020.0
24771     4080.0
25559     1140.0
26248     1440.0
26264       60.0
26374     1020.0
26510     1560.0
26694     1920.0
26762     1320.0
26800     8940.0
26817     3420.0
26977     1020.0
27160     2700.0
27400     3720.0
27574    14340.0
27595     1740.0
27643     1320.0
27805      900.0
27828     4080.0
27846     5340.0
27860    18360.0
28408     8460.0
28698    10114.0
28725     9900

In [177]:
unique_tickets["HoldCount"][unique_tickets["HoldCount"]>0]

Series([], Name: HoldCount, dtype: int64)

In [174]:
dfmerged[dfmerged["HoldDuration"]==10114]

,TicketNumber,IncidentId,Created_On,Receiveddate,ResolvedDate,Queue,StatusReason,Priority,ValidCase,BusinessFunction,...,HoldTypeName_3rd Party,HoldTypeName_Customer,HoldTypeName_Internal,AssignedToGroup_BPO,AssignedToGroup_CRMT,AssignedToGroup_Internal,AssignedToGroup_Microsoft IT,AssignedToGroup_Ops. Program Manager,AssignedToGroup_Submitter (Contact),HoldCount
28698,898770,FCD7312B-6E52-E711-80F7-3863BB3C66F8,2017-06-16 08:31:27,2017-06-16 08:30:00,2017-06-16 09:49:03,<VL EOC Open>,Problem Solved,Normal,1,Signature to Invoice (S2I),...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [74]:
# ticket = "5-0000000402882"
# int(ticket)
# newstr = oldstr.replace("M", "")



       TicketNumber  HoldCount  HoldDuration  HoldTypeName_3rd Party  \
0            464223          0             0                       0   
1            418488          0             0                       0   
2            402148          0             0                       0   
3            463272          0             0                       0   
4            408448          0             0                       0   
5            452541          0             0                       0   
6            463528          0             0                       0   
7            398020          0             0                       0   
8            400452          0             0                       0   
9            401479          0             0                       0   
10           401587          0             0                       0   
11           363780          0             0                       0   
12           401525          0             0                    

In [53]:
# unique_tickets = pd.DataFrame(df["TicketNumber"].unique().tolist(), columns=["TicketNumber"])
single_ticket = ["5-0000000401525"]
for ticket in single_ticket:
    givenvlue = dfdummies[dfdummies["TicketNumber"]==ticket].sum()["HoldTypeName_3rd Party"]
    givennovalue = dfdummies[dfdummies["TicketNumber"]==ticket].sum()
print(type(givennovalue))
print(givenvlue, givennovalue)


<class 'pandas.core.series.Series'>
0 TicketNumber                                                 5-0000000401525
ActivityId                              E35F3716-8507-E711-80E9-3863BB2ED230
RegardingObjectId                       5CE3D229-ABFA-E611-80F1-FC15B428DE24
HoldDuration                                                            1440
Statuscode                                                         Completed
Statecode                                                          Completed
TimeZoneRuleVersionNumber                                                  0
Reason                                            Incorrect Document Version
StartTime                                                2017-03-13 00:36:29
EndTime                                                  2017-03-14 00:30:20
IsThreeStrikeRule                                                          0
HoldSubReason                                                           None
Modified_On                           

In [50]:
tickets = ["5-0000000452541"]
for ticket in tickets:
    summed = dfdummies[dfdummies["TicketNumber"]==ticket].sum()["HoldTypeName_3rd Party"]
#     counted = dfdummies[dfdummies["TicketNumber"]==ticket].count()["HoldTypeName_3rd Party"]
#     print(dfdummies[dfdummies["TicketNumber"]==ticket])
print(summed)

2


In [30]:
dfdummies = pd.get_dummies(data=df, columns=["HoldTypeName", "AssignedToGroup"])
dfdummies

,TicketNumber,ActivityId,RegardingObjectId,HoldDuration,Statuscode,Statecode,TimeZoneRuleVersionNumber,Reason,StartTime,EndTime,...,Deleted,HoldTypeName_3rd Party,HoldTypeName_Customer,HoldTypeName_Internal,AssignedToGroup_BPO,AssignedToGroup_CRMT,AssignedToGroup_Internal,AssignedToGroup_Microsoft IT,AssignedToGroup_Ops. Program Manager,AssignedToGroup_Submitter (Contact)
0,5-0000000464223,28E0D3AE-0907-E711-80E9-3863BB2ED230,2058B401-7A06-E711-80F3-FC15B428DE24,4620,Completed,Completed,NaN,Trade - Trade Status Hold,2017-03-12 09:53:03,2017-03-15 14:16:17,...,NaN,1,0,0,0,0,1,0,0,0
1,5-0000000418488,79A45B63-3507-E711-80E9-3863BB2ED230,54A3CC57-50FE-E611-80F1-FC15B428DE24,0,Open,Open,NaN,Price Error,2017-03-12 15:05:55,NaN,...,NaN,0,1,0,0,0,0,0,0,1
2,5-0000000402148,241CB04D-3907-E711-80E9-3863BB2ED230,60D07540-BBFA-E611-80F6-FC15B4289B54,120,Completed,Completed,0.0,PO Number Error,2017-03-12 15:33:56,2017-03-12 17:18:02,...,NaN,0,1,0,0,0,0,0,0,1
3,5-0000000463272,996E4A98-5407-E711-80E9-3863BB2ED230,905C98C5-CB05-E711-80F3-FC15B4284D68,3300,Completed,Completed,0.0,Internal Hold - Team Lead,2017-03-12 18:49:25,2017-03-15 01:46:18,...,NaN,0,0,1,0,0,0,0,0,1
4,5-0000000408448,2A1CD3FE-5907-E711-80E9-3863BB2ED230,51E16092-0BFD-E611-80F1-FC15B428DE24,180,Completed,Completed,NaN,A - Missing information from customer - Confir...,2017-03-12 19:28:00,2017-03-12 22:42:43,...,NaN,0,0,1,0,0,0,0,0,1
5,5-0000000452541,DB8A8576-6A07-E711-80E9-3863BB2ED230,FA90A09E-4904-E711-80F4-C4346BAC89C4,8400,Completed,Completed,0.0,Field - Waiting for Clarification,2017-03-12 21:25:54,2017-03-18 17:50:12,...,NaN,1,0,0,0,0,0,0,0,1
6,5-0000000463528,6EE062A1-7207-E711-80E9-3863BB2ED230,A230ACF1-D605-E711-80F3-FC15B4284D68,4560,Completed,Completed,0.0,PO Number Error,2017-03-12 22:24:24,2017-03-16 02:24:03,...,NaN,0,1,0,0,0,0,0,0,1
7,5-0000000398020,4A923146-7507-E711-80E9-3863BB2ED230,DC5CF67B-17FA-E611-80F1-FC15B428DE24,8580,Completed,Completed,0.0,Incorrect Document Version,2017-03-12 22:43:21,2017-03-18 21:15:16,...,NaN,0,1,0,0,0,0,0,0,1
8,5-0000000400452,C798697D-7507-E711-80E9-3863BB2ED230,5C68CEF7-89FA-E611-80F6-FC15B4289B54,1320,Completed,Completed,0.0,Incorrect Document Version,2017-03-12 22:44:46,2017-03-13 20:08:41,...,NaN,0,1,0,0,0,0,0,0,1
9,5-0000000401479,07BCEC1E-7607-E711-80E9-3863BB2ED230,CD32F259-AAFA-E611-80F6-FC15B4289B54,61736,Open,Open,0.0,PO Number Error,2017-03-12 22:49:17,NaN,...,NaN,0,1,0,0,0,0,0,0,1


In [25]:
for index, row in df.iterrows():
   ....:     print row['c1'], row['c2']

TicketNumber
ActivityId
RegardingObjectId
HoldDuration
Statuscode
Statecode
TimeZoneRuleVersionNumber
HoldTypeName
Reason
AssignedToGroup
StartTime
EndTime
IsThreeStrikeRule
HoldSubReason
Modified_On
Deleted
